In [1]:
##Load metadata from file

import ast

d = {}
nary = {}
with open("metadata_trial4.txt", 'r') as f:
    for line in f:
        d = (ast.literal_eval(line))
        nary.update({d['id'] : d})

In [2]:
#Filter terms that have children and aspect

filnary = {}
kilnary = {}
nochild = {}

for key, value in nary.items():
    if value.get('children') != None:
        filnary = ({'id' : value['id'], 'name' : value['name'], 'aspect' : value['aspect'], 
                    'children' : value['children']})
        kilnary.update({key : filnary})
    else:
        filnary = ({'id' : value['id'], 'name' : value['name'], 'aspect' : value['aspect']})
        nochild.update({key : filnary})

In [3]:
#remove anything that's not a biological process or molecular function from kilnary

keys = []
for key, value in kilnary.items():
    if value['aspect'] != 'biological_process' and value['aspect'] != 'molecular_function':
        keys.append(key)
        
for i in keys:
    del kilnary[i]

In [4]:
#get parents

for key, value in kilnary.items():
    for item in value['children']:
        if nary[item['id']].get('parent') == None:
            nary[item['id']]['parent'] = []
            nary[item['id']]['parent'].append({'id' : key, 'relation' : item['relation']})
        else:
            nary[item['id']]['parent'].append({'id' : key, 'relation' : item['relation']})
            
for key, value in nary.items():
    if value.get('parent') == None:
        value['parent'] = []

In [5]:
#import owlready2

import types
from owlready2 import *
onto = get_ontology("http://test.org/onto.owl")

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


In [6]:
#classmaker function to make FTC classes

def classmaker(term, parentclass):
                classnameA = 'GOA' + str(term)
                classnameP = 'GOP' + str(term)
                with onto:
                    classname = types.new_class(classnameA, (parentclass,))
                    classname = types.new_class(classnameP, (parentclass,))
                return 0

In [7]:
#define core classes

with onto:
    class therapeutic_agent(Thing):
        namespace = onto
    class biological_process(Thing):
        namespace = onto
    class molecular_function(Thing):
        namespace = onto
    class drug(Thing):
        namespace = onto

In [8]:
#recursive classmakers


def recursive_classmaker_down(term, parent):
    parent = getattr(onto, parent)
    with onto:
        nc = types.new_class(term, (parent,))
    parent = getattr(onto, term)
    for item in nary[term]['children']:
        recursive_classmaker_down(item['id'][3:], parent)
    return 0



def recursive_classmaker_up(term, parent):
    if parent not in [str(s) for s in list(onto.classes())]:
        for item in nary[('GO:' + term)]['parent']:
            recursive_classmaker_up(parent, item['id'][3:])
            
    if nary[('GO:' + term)]['parent'] == []:
        with onto:
            nc = types.new_class(parent, (biological_process,))
        #recursive_classmaker_down(term, parent)
    return 0



In [ ]:
#Using classmaker to make FTC classes

for key, value in kilnary.items():
    if value['aspect'] == 'biological_process':
        for item in value['children']:
            if item['relation'] == 'positively_regulates' or 'negatively_regulates' or 'regulates':
                recursive_classmaker_up(str(item['id'][3:]), value['id'][3:])
            if item['relation'] == 'capable of' or 'part_of' or 'capable_of_part_of' or 'has_part':
                recursive_classmaker_up(str(item['id'][3:]), value['id'][3:])
            
            
            
#             if item['relation'] == 'negatively_regulates':
#                 recursive_classmaker_up(str(item['id'][3:]), value['id'][3:])
#             if item['relation'] == 'regulates':
#                 recursive_classmaker_up(str(item['id'][3:]), value['id'][3:])
#     if value['aspect'] == 'molecular_function':
#         for item in value['children']:
#             if item['relation'] == 'positively_regulates':
#                 classmaker(str(item['id'][3:]), therapeutic_agent)
#             if item['relation'] == 'negatively_regulates':
#                 classmaker(str(item['id'][3:]), therapeutic_agent)
#             if item['relation'] == 'regulates':
#                 classmaker(str(item['id'][3:]), therapeutic_agent)


In [ ]:
onto.save(file = "testowl2", format = "rdfxml")

In [11]:
list(onto.classes())

[onto.therapeutic_agent,
 onto.biological_process,
 onto.molecular_function,
 onto.drug,
 onto.0008150,
 onto.0007582,
 onto.0000004,
 onto.0044699,
 onto.0008152,
 onto.0044236,
 onto.0044710,
 onto.0071704,
 onto.0009987,
 onto.0050875,
 onto.0044763,
 onto.0008151,
 onto.0043170,
 onto.0044237,
 onto.0043283,
 onto.0044259,
 onto.0006807,
 onto.0006725,
 onto.0034641,
 onto.0044238,
 onto.0046483,
 onto.1901360,
 onto.0006139,
 onto.0055134,
 onto.0044260,
 onto.0090304,
 onto.0034960,
 onto.0050896,
 onto.0051869,
 onto.0006950,
 onto.0051716,
 onto.0033554,
 onto.0006259,
 onto.0006974,
 onto.0055132,
 onto.0034984,
 onto.0065007,
 onto.0050789,
 onto.0050791,
 onto.0019222,
 onto.0050794,
 onto.0044246,
 onto.0051244,
 onto.0031323,
 onto.0051171,
 onto.0080090,
 onto.0019219,
 onto.0060255,
 onto.0048583,
 onto.0080134,
 onto.0080135,
 onto.0006281,
 onto.0051052,
 onto.2001020,
 onto.0048519,
 onto.0043118,
 onto.0009892,
 onto.0044252,
 onto.0048523,
 onto.0051243,
 onto.00313